In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import statistics
import scipy.stats as st


import time
start = time.time()

from scipy.stats import linregress
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
filepath='/home/yanchao/COVID19/Data/'
EdgeCutoff=300
Mob=pd.read_csv(filepath+'MSA_Dataset_TillJune.csv',index_col=0)

In [ ]:
CtyConfOri=pd.read_csv(filepath+'ArrivalTime/'+'County_ArrT_allrecords_Th1.csv')
CtyConfOri['FIPS']=CtyConfOri['FIPS'].astype(int)

In [ ]:
CtySource=6075
CtySourcelist=[6075, 25025]


print(CtySourcelist)
print(CtyConfOri[CtyConfOri['FIPS'].isin(CtySourcelist)][['Admin2','Province_State']])
Ctylist=list((set(dict.fromkeys(list(Mob['GEOID1'])))) & (set(dict.fromkeys(list(CtyConfOri['FIPS'])))))
CtyM=len(Ctylist)

CtyTargetlist=[i for i in Ctylist if i not in CtySourcelist] 

In [ ]:
CtyDayZero=CtyConfOri[CtyConfOri['FIPS']==CtySource]['Days'].iat[0]
CtyConfOri['Daysfrom122']=CtyConfOri['Days']
CtyConfOri['Days']=CtyConfOri['Daysfrom122']-CtyDayZero

In [ ]:
datelistori=list(dict.fromkeys(Mob['date']))
Ctydatelist=datelistori[int(CtyDayZero):] #[0:52]

In [ ]:
SingleEffdist=pd.DataFrame()
MultiEffdist=pd.DataFrame()
MixEffdist=pd.DataFrame()
Days=-int(CtyDayZero)
sourcelist=CtySourcelist
targetlist=CtyTargetlist

for date in datelistori:
    DayMob=pd.DataFrame()
    DayMob=Mob[Mob['date']==date]

    DayMob['Fraction']=DayMob['counts_x']/DayMob['counts_y'] #sum_x, Basesum_x
    DayMob['effdist']=1-np.log(DayMob['Fraction'])
    G=nx.from_pandas_edgelist(DayMob,source='GEOID1',target='GEOID2',edge_attr='effdist',create_using=nx.DiGraph())

    d={'Target':targetlist}
    dftemp=pd.DataFrame(d)
    length,path = nx.single_source_dijkstra(G,CtySource,cutoff=EdgeCutoff,weight='effdist')
    dftemp['CapD']=dftemp['Target'].map(length)
    dftemp['Date']=date
    dftemp['Days']=Days
    dftemp['Route']=dftemp['Target'].map(path)
    dftemp=dftemp.dropna(subset=['Route'])
    dftemp['Nodes']=dftemp['Route'].apply(lambda x: len(x))
    SingleEffdist=SingleEffdist.append(pd.DataFrame(dftemp))
    Days=Days+1
     
Days=-int(CtyDayZero)
for date in datelistori:
    DayMob=pd.DataFrame()
    DayMob=Mob[Mob['date']==date]
    DayMob['Fraction']=DayMob['counts_x']/DayMob['counts_y'] #sum_x
    DayMob['effdist']=1-np.log(DayMob['Fraction'])
    G=nx.from_pandas_edgelist(DayMob,source='GEOID1',target='GEOID2',edge_attr='effdist',create_using=nx.DiGraph())
    dftemp=pd.DataFrame()
    for source in sourcelist:
        d={'Source':source,'Target':targetlist}
        dftemprec=pd.DataFrame(d)
        length,path = nx.single_source_dijkstra(G,source,cutoff=EdgeCutoff,weight='effdist')
        dftemprec['effdist']=dftemprec['Target'].map(length)
        dftemp=dftemp.append(dftemprec)
    dftemp['suminvexp']=1/np.exp(dftemp['effdist'])
    dftemp=dftemp.groupby('Target').sum()
    dftemp['suminvexp'].replace(0, np.nan, inplace=True)
    dftemp=dftemp.reset_index()[['Target','suminvexp']]
    dftemp['CapD']=np.log(CtyM)-np.log(dftemp['suminvexp'])
    dftemp=dftemp.drop(columns=['suminvexp'])
    dftemp['Date']=date
    dftemp['Days']=Days
    dftemp['Nodes']=np.log(CtyM)
    MultiEffdist=MultiEffdist.append(pd.DataFrame(dftemp))
    Days=Days+1

SingleEffdistClean=SingleEffdist.drop(columns=['Route','Nodes'])
SingleEffdistClean['Src']='Single'
MultiEffdistClean=MultiEffdist.drop(columns=['Nodes'])
MultiEffdistClean['Src']='Multiple'
MixEffdist=SingleEffdistClean.append(MultiEffdistClean)
#MixEffdist
MixEffdist.to_csv('/Data/Src'+str(CtySourcelist[0])+'_'+str(CtySourcelist[1])+'_EF_LN_NoBase_DevCnt_AllCty_AllTime_tillJune.csv')